In [ ]:
import requests as rq
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time

import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

import stanza
import nltk

from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

## парсинг

In [ ]:
main = []
for num in range(1, 1069):
    main.append(f'https://shikimori.one/animes/page/{num}')

In [ ]:
page = rq.get(main[0], headers={'User-Agent': UserAgent().chrome})
print(page.status_code)
soup = BeautifulSoup(page.text, features="html.parser")

In [ ]:
links = [url.get('href') for url in soup.find_all('a')]

page_links = []
for link in links:
    if 'https://shikimori.one/animes/' in str(link) and 'https://shikimori.one/animes/page/' not in str(link):
        page_links.append(link)

In [ ]:
def collect_links(list):
    mistakes = []
    page_links = []

    for i in list:
        page = rq.get(i, headers={'User-Agent': UserAgent().chrome})
        if page.status_code != 200:
            mistakes.append(i)
        else:
            soup = BeautifulSoup(page.text, features="html.parser")
            links = [url.get('href') for url in soup.find_all('a')]

            for link in links:
                if 'https://shikimori.one/animes/' in str(link) and 'https://shikimori.one/animes/page/' not in str(link):
                    page_links.append(link)
    
    return [mistakes, page_links]

In [ ]:
all_mistakes = []
all_page_links = []

mistakes, page_links = collect_links(main)

all_mistakes.extend(mistakes)
all_page_links.extend(page_links)

In [ ]:
while all_mistakes:
    new_mistakes, new_page_links = collect_links(all_mistakes)
    all_mistakes = []
    all_mistakes.extend(new_mistakes)
    all_page_links.extend(new_page_links)

In [ ]:
all_page_links = list(set(all_page_links))
len(all_page_links)

In [ ]:
with open('links.txt', 'x') as file:
    print(*all_page_links, file=file, sep='\n')

In [ ]:
# функция-парсер

def get_info(link):
    page = rq.get(link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(page.text, features="html.parser")
    
    # название
    try:
        name_ru, name_jp = soup.find('h1').text.split(' / ')
    except:
        name_jp = soup.find('h1').text
        name_ru = None
    
    # описание
    try:
        description = soup.find('div', class_='c-description').find('div', class_='b-text_with_paragraphs').text
    except:
        description = None

    # рейтинг
    try:
        rating = soup.find('div', class_='c-info-right').find('div', class_='scores').find(itemprop='ratingValue').get('content')
    except:
        rating = None
        
    # тип
    info = soup.find('div', class_='c-about').find_all('div', class_='value')
    type = info[0].text

    # студия
    links = [url.get('href') for url in soup.find_all('a')]
    studio = None
    for l in links:
        if 'studio/' in str(l):
            studio = str(l).split('studio/')[1]

    # жанры
    genre_check, genres = [], []
    for i in info:
        genre_check.extend(i.find_all('span', class_='genre-ru'))
    for j in genre_check:
        genres.append(j.text.lower())
    
    #кол-во и длина эпизодов 
    ep_len = soup.find('div', class_='c-about').find('span', class_=None).text
    try:
        ep = int(info[1].text)
    except:
        if '/' in info[1].text:
            ep = info[1].text
        else:
            ep = 1
    
    # статус
    ongoing = True if '/' in info[1].text else False

    # возрастные ограничения, дата выхода и окончания
    date_age = soup.find('div', class_='c-about').find_all('span', class_='b-tooltipped dotted mobile unprocessed')
    
    try:
        if len(date_age) == 1:
            age_rating = date_age[0].text
            date = info[4].text if ongoing else info[2].text
        else:
            date, age_rating = date_age[0].text, date_age[1].text
    except:
        date, age_rating = None, None
    
    try:
        if 'по' in date:
            start_date, end_date = date.split(' по ')
            start_date = start_date.replace('\xa0с ', '')
        elif '-' in date:
            start_date, end_date = date.split('-')
            start_date = start_date.replace('\xa0в ', '')
            end_date = end_date.replace(' гг.', '')
        else:
            #start_date = date.replace('\xa0', '')
            start_date = date.replace('\xa0с ', '') if ongoing == True else date.replace('\xa0', '')
            end_date = None
    except:
        start_date = None
        end_date = None


    information = {
        'name_ru': name_ru,
        'name_jp': name_jp,
        'type': type,
        'ep': ep,
        'ep_len': ep_len,
        'start_date': start_date,
        'end_date': end_date,
        'ongoing': ongoing,
        'genres': genres,
        'age_rating': age_rating,
        'studio': studio,
        'rating': rating,
        'description': description}
    
    return information

In [ ]:
with open('links.txt', 'r') as file:
    links = file.read()

links_to_parse = links.split('\n')

mistakes = []
parsed = []

for link in links_to_parse:
    try:
        parsed.append(get_info(link))
    except:
        mistakes.append(link)
    time.sleep(0.5)

In [ ]:
df_1 = pd.DataFrame(parsed)

In [ ]:
# помечено как 18+ и заблокировано - добавляю информацию вручную
# описание из манги (первоисточника сериала) с этого же сайта: https://shikimori.one/mangas/z21-death-note

death_note = {
        'name_ru': 'Тетрадь смерти',
        'name_jp': 'Death Note',
        'type': 'TV Сериал',
        'ep': 37,
        'ep_len': '23 мин.',
        'start_date': 2006,
        'end_date': 2007,
        'ongoing': False,
        'genres': ['сёнен', 'сверхъестественное', 'триллер', 'психологическое'],
        'age_rating': 'R-17',
        'studio': '11-Madhouse',
        'rating': '8.62',
        'description': 'Лайт Ягами — образцовый 17-летний выпускник, баллы за экзамены которого находятся в первых строках рейтинга всей Японии. Сидя на уроке, он замечает, что за окном что-то упало. На перемене он поднимает загадочный предмет и им оказывается черная тетрадь с надписью «Тетрадь смерти». Внутри была инструкция по использованию: "Человек, имя которого будет записано в тетради, умрет". Имея свои взгляды на систему наказания, Лайт решает установить собственное правосудие, использовать тетрадь для «очищения» мира от зла — убивать преступников. Когда действия Лайта становятся заметны для мирового правительства, на след неуловимого «Киры» (так мир окрестил нового мессию, решившего искоренить зло на планете) выходит детектив мирового класса, называющий себя «L», который поставил себе целью разоблачить убийцу. Так начинается одно из самых психологичных, напряженных и сильных противостояний в истории японской манги, величайшая битва умов.'}

In [ ]:
del mistakes[5]

In [ ]:
# в остальных 6 случаях ошибка возникала из-за отсутствия информации о длительности эпизода
# ниже та же функция, но длительность эпизода сразу задана None

def get_info_mistakes(link):
    page = rq.get(link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(page.text, features="html.parser")
    
    # название
    try:
        name_ru, name_jp = soup.find('h1').text.split(' / ')
    except:
        name_jp = soup.find('h1').text
        name_ru = None
    
    # описание
    try:
        description = soup.find('div', class_='c-description').find('div', class_='b-text_with_paragraphs').text
    except:
        description = None

    # рейтинг
    try:
        rating = soup.find('div', class_='c-info-right').find('div', class_='scores').find(itemprop='ratingValue').get('content')
    except:
        rating = None
        
    # тип
    info = soup.find('div', class_='c-about').find_all('div', class_='value')
    type = info[0].text

    # студия
    links = [url.get('href') for url in soup.find_all('a')]
    studio = None
    for l in links:
        if 'studio/' in str(l):
            studio = str(l).split('studio/')[1]

    # жанры
    genre_check, genres = [], []
    for i in info:
        genre_check.extend(i.find_all('span', class_='genre-ru'))
    for j in genre_check:
        genres.append(j.text.lower())
    
    #кол-во и длина эпизодов 
    ep_len = None
    try:
        ep = int(info[1].text)
    except:
        if '/' in info[1].text:
            ep = info[1].text
        else:
            ep = 1
    
    # статус
    ongoing = True if '/' in info[1].text else False

    # возрастные ограничения, дата выхода и окончания
    date_age = soup.find('div', class_='c-about').find_all('span', class_='b-tooltipped dotted mobile unprocessed')
    
    try:
        if len(date_age) == 1:
            age_rating = date_age[0].text
            date = info[4].text if ongoing else info[2].text
        else:
            date, age_rating = date_age[0].text, date_age[1].text
    except:
        date, age_rating = None, None
    
    try:
        if 'по' in date:
            start_date, end_date = date.split(' по ')
            start_date = start_date.replace('\xa0с ', '')
        elif '-' in date:
            start_date, end_date = date.split('-')
            start_date = start_date.replace('\xa0в ', '')
            end_date = end_date.replace(' гг.', '')
        else:
            start_date = date.replace('\xa0с ', '') if ongoing == True else date.replace('\xa0', '')
            end_date = None
    except:
        start_date = None
        end_date = None


    information = {
        'name_ru': name_ru,
        'name_jp': name_jp,
        'type': type,
        'ep': ep,
        'ep_len': ep_len,
        'start_date': start_date,
        'end_date': end_date,
        'ongoing': ongoing,
        'genres': genres,
        'age_rating': age_rating,
        'studio': studio,
        'rating': rating,
        'description': description}
    
    return information

In [ ]:
parsed_2 = [get_info_mistakes(page) for page in mistakes]

In [ ]:
parsed_2.append(death_note)
df_2 = pd.DataFrame(parsed_2)
df_2['start_date'][2:5] = None

In [ ]:
# я решила добавить ссылки (к сожалению, не догадалась добавить это в функцию сразу)

links_2 = ['https://shikimori.one/animes/56624-araburu-kisetsu-no-otome-domo-yo-mini-anime',
'https://shikimori.one/animes/55566-wasted-chef',
'https://shikimori.one/animes/49941-gundam-uc-x-nike-sb',
'https://shikimori.one/animes/42748-pure-shield',
'https://shikimori.one/animes/33312-color-noise',
'https://shikimori.one/animes/54160-future-kid-takara',
'https://shikimori.one/animes/1535-death-note']

df_2['link'] = links_2

links_1 = [l for l in links_to_parse if l not in links_2]
df_1['link'] = links_1

In [ ]:
df = pd.concat([df_1, df_2])
df.to_csv('anime_info.csv', sep = ',', encoding='utf-8')

## очистка данных

In [ ]:
df = pd.read_csv('anime_info.csv')

In [ ]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df = df[df['type'] != 'Реклама']
df = df[df['type'] != 'Проморолик']
df = df.dropna(subset = ['description'])

In [ ]:
# год начала выхода

start_date = df['start_date'].fillna(0).tolist()
start_year = []
start_year_mistakes = []

for i in range(len(start_date)):
    try:
        res = int(re.search(r'\d{4}', start_date[i]).group())
        start_year.append(res)
    except:
        start_year.append(None)
        start_year_mistakes.append(i)

In [ ]:
# год конца выхода

ep = df['ep'].tolist()
ongoing = df['ongoing'].tolist()

end_date = df['end_date'].fillna(0).tolist()
end_year = []
end_year_mistakes = []


for i in range(len(end_date)):
    if end_date[i] == 0:
        if ep[i] in [1, '1']:
            end_year.append(start_year[i])
        elif ongoing[i] == True:
            end_year.append(2024)
        else:
            end_year.append(None)
            end_year_mistakes.append(i)
    else:
        try:
            res = int(re.search(r'\d{4}', end_date[i]).group())
            end_year.append(res)
        except:
            end_year.append(None)
            end_year_mistakes.append(i)

In [ ]:
df['start_date'] = start_year 
df['end_date'] = end_year

In [ ]:
# код для ручной проверки дат, которые попали в ошибки

all_date_mistakes = list(set(end_year_mistakes) | set(start_year_mistakes))

all_date_clean = pd.DataFrame(start_year)
all_date_clean['1'] = end_year

df_date_check = df.iloc[all_date_mistakes, [0, 5, 6, 13]]
df_date_check['start_date_new'] = all_date_clean.iloc[all_date_mistakes, 0]
df_date_check['end_date_new'] = all_date_clean.iloc[all_date_mistakes, 1]
df_date_check['index'] = all_date_mistakes

df_date_check.to_excel('date_check.xlsx')

date_check_links = df_date_check['link'].tolist()

with open('date_check_links.txt', 'x') as file:
    print(*date_check_links, file=file, sep='\n')

In [ ]:
# у многих работ указана дата начала, поэтому попробую спарсить еще раз

parsing_date_again = []
for d in date_check_links:
    page = rq.get(d, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(page.text, features="html.parser")
    try:
        date = soup.find('div', class_='c-about').find_all('div', class_='value')[3].text
        parsing_date_again.append(int(re.search(r'\d{4}', date).group()))
    except:
        parsing_date_again.append(None)
    time.sleep(0.4)

In [ ]:
# так как в список ошибок попали в основном короткие работы, год конца выхода будет таким же, как и год начала

df_date_clean = df.iloc[all_date_mistakes]
df_date_clean['start_date'] = parsing_date_again
df_date_clean['end_date'] = parsing_date_again

In [ ]:
df.drop(df.index[all_date_mistakes], axis=0, inplace=True)
df_upd = pd.concat([df, df_date_clean])

In [ ]:
# студия

studio_dirty = df_upd['studio'].tolist()
studio_clean = []

for i in range(len(studio_dirty)):
    try:
        studio_clean.append(re.sub(r'\d{1,}-', '', studio_dirty[i]))
    except:
        studio_clean.append(studio_dirty[i])

df_upd['studio'] = studio_clean

In [ ]:
df_upd.to_csv('anime_info_upd.csv', sep = ',', encoding='utf-8')

In [ ]:
df = pd.read_csv('anime_info_upd.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# кол-во эпизодов: изменение формата для онгоингов с "12 / 24" или "100 / ?" на одно число

ep_dirty = df['ep'].tolist()
ep_clean = []
ep_mistakes = []

for i in range(len(ep_dirty)):
    try:
        res = int(ep_dirty[i])
    except:
        if df.iloc[i, 7] == True:
            res = int(ep_dirty[i].split(' / ')[0]) if '?' in ep_dirty[i] else int(ep_dirty[i].split(' / ')[1])
        else:
            res = ep_dirty[i]
            ep_mistakes.append(i)
    ep_clean.append(res)

df['ep'] = ep_clean

In [ ]:
# возрастной рейтинг - в некоторые строки попали даты

age_dirty = df['age_rating'].fillna(0).tolist()
age_na = []
age_mistakes = []

for i in range(len(age_dirty)):
    if age_dirty[i] not in ['G', 'PG', 'PG-13', 'R+', 'R-17']:
        if age_dirty[i] == 0:
            age_na.append(i)
        else:
            age_mistakes.append(i)

In [ ]:
df_age_na = df.iloc[age_na]
df_age = df.iloc[age_mistakes]

age_date = df_age['age_rating'].tolist()
date_new = [int(re.search(r'\d{4}', i).group()) for i in age_date]

In [ ]:
# (1) год либо не указан, (2) либо указан правильно у работ с одинаковым годом начала и конца, (3) либо год конца указан неправильно (такой же, как год начала)
# ошибку в третьем случае исправлю ниже

df_age['start_date'], df_age['end_date'] = date_new, date_new

In [ ]:
names_ru = ['Викторина Токико', 'Волшебная улица Чанъань', 'Записи о даосском мече дождя и ветра', 'Обезьяний пик', 'Волейбольный клуб старшей школы Сэйин: Мини-аниме — Курсы по волейболу']

manual_check = df_age.loc[df_age['name_ru'].isin(names_ru)]

years = [2021, 2021, 2019, 2018, 2021]
manual_check['end_date'] = years

In [ ]:
# убираю строки, которые исправила выше, и добавляю эти же исправленные строки

index = df_age[df_age['name_ru'].isin(names_ru)].index
df_age.drop(index, inplace=True)
df_age = pd.concat([df_age, manual_check])

In [ ]:
# попробую еще раз спарсить возрастной рейтинг

df_wrong_age = pd.concat([df_age, df_age_na])
links = df_wrong_age['link'].tolist()

parsing_age_again = []
none_links = []
mistakes_links = []

for l in links:
    page = rq.get(l, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(page.text, features="html.parser")
    try:
        age = soup.find('div', class_='c-about').find('span', class_='b-tooltipped dotted mobile unprocessed').text
        if age in ['G', 'PG', 'PG-13', 'R+', 'R-17']:
            parsing_age_again.append(age)
        else:
            parsing_age_again.append(None)
            mistakes_links.append(l)
    except:
        parsing_age_again.append(None)
        none_links.append(l)
    time.sleep(0.4)

In [ ]:
# я проверила эти ссылки вручную - возрастной рейтинг действительно не указан

with open('age_na_links.txt', 'x') as file:
    print(*none_links, file=file, sep='\n')

# у работ, из которых вместо рейтинга подтягивалась дата, тоже не указан рейтинг

with open('age_mistakes_links.txt', 'x') as file:
    print(*mistakes_links, file=file, sep='\n')

In [ ]:
df_wrong_age['age_rating'] = parsing_age_again

In [ ]:
# функция для замены части датасета

def replace_rows(df_main, df_new_rows):
    names = df_new_rows['name_jp'].tolist()
    index = df_main[df_main['name_jp'].isin(names)].index
    df = df_main.drop(index)
    return pd.concat([df, df_new_rows])

In [ ]:
df_upd = replace_rows(df, df_wrong_age)
df_upd.to_csv('anime_info_upd_2.csv', sep = ',', encoding='utf-8')

In [ ]:
df = pd.read_csv('anime_info_upd_2.csv', converters={'genres': pd.eval})
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# в колонке с длительностью эпизода должно остаться только общее количество минут

df['ep_len'] = np.where((df.ep_len == '···'), None, df.ep_len)
ep_len = df['ep_len'].fillna('').tolist()
ep_len_new = []
ep_len_mistakes = []

for i in range(len(ep_len)):
    if 'час' in ep_len[i] and 'мин' in ep_len[i]:
        res = 0
        hour, min = ep_len[i].split(' ч')
        min = int(re.search(r'\d+', min).group())
        res += int(hour) * 60 + min
        ep_len_new.append(res)
    elif 'час' in ep_len[i]:
        res = 0
        hour = ep_len[i].split(' ч')
        res += int(hour[0]) * 60
        ep_len_new.append(res)
    else:
        try:
            res = int(re.search(r'\d+', ep_len[i]).group())
            ep_len_new.append(res)
        except:
            ep_len_new.append(None)
            ep_len_mistakes.append(i)

In [ ]:
# проверила эти ссылки вручную - длительность эпизода нигде не указана, в датасете останется None

links = df.iloc[ep_len_mistakes]['link'].tolist() 
with open('ep_len_links.txt', 'x') as file:
    print(*links, file=file, sep='\n')

In [ ]:
df['ep_len'] = ep_len_new

In [ ]:
# у всех онгоингов год конца выхода должен быть указан 2024

df_ongoing = df.loc[df['ongoing'] == True]
df_ongoing['end_date'] = 2024
df = replace_rows(df, df_ongoing)

In [ ]:
# у некоторых работ не указано название на русском
# сначала исправлю вручную то, что не потребуется переводить

df.loc[df['name_jp'] == 'Детектив Холмс: Дело о Голубом Рубине/Дело о сокровищах со дна моря / Meitantei Holmes: Aoi Ruby no Maki / Kaitei no Zaihou no Maki', 'name_jp'] = 'Meitantei Holmes: Aoi Ruby no Maki / Kaitei no Zaihou no Maki'
df.loc[df['name_jp'] == 'Meitantei Holmes: Aoi Ruby no Maki / Kaitei no Zaihou no Maki', 'name_ru'] = 'Детектив Холмс: Дело о Голубом Рубине/Дело о сокровищах со дна моря'

df.loc[df['name_jp'] == '663114', 'name_ru'] = '663114'

In [ ]:
# теперь соберу ссылки на оставшиеся работы без названия на русском, переведу названия в онлайн-переводчике

df_no_name = df.loc[df['name_ru'].isna()]
links = df_no_name['link'].tolist()
with open('no_name_links.txt', 'x') as file:
    print(*links, file=file, sep='\n')

In [ ]:
# когда я проверяла ссылки, в описании одной из работ прочитала, что это реклама, поэтому удаляю

df_no_name.drop(df_no_name.loc[df_no_name['name_jp'] == 'Wares: Beyond'].index, inplace=True)
df.drop(df.loc[df['name_jp'] == 'Wares: Beyond'].index, inplace=True)

In [ ]:
df_no_name['name_jp'].tolist()

In [ ]:
names_ru_translated = ['Стоп! Игра с зажигалкой: Команда тушения деревни животных Дейдо',
'Третий лишний', 
'Безопасность дорожного движения в городе Синсэнгуми Отасукегуми Аймедзаси-тай. Стремитесь соблюдать правила дорожного движения! Зоопарк!',
'Опетте',
'Издевательства - это абсолютно неправильно!',
'Гимнастика в соответствии с руководством по скринингу опорно-двигательного аппарата',
'Исполнительный комитет',
'Я немедленно сбежал: чему меня научило Великое землетрясение на востоке Японии',
'Матиерика',
'Все в другом мире — призраки!',
'Знаете ли вы, как это больно: я не могу простить! Кибербуллинг',
'Положи руку на грудь',
'Начало дружбы',
'Радужная связь',
'Нана Мун',
'Животные под угрозой исчезновения',
'Однокрылый механизм Хроноса']

df_no_name['name_ru'] = names_ru_translated
df = replace_rows(df, df_no_name)

In [ ]:
df.reset_index(drop=True, inplace=True)

index_drop = []
for i in range(len(df)):
    if df.loc[i].isna().sum() > 3:
        index_drop.append(i)

df_upd = df.drop(index_drop, axis=0)

In [ ]:
df_upd['start_date'] = df_upd['start_date'].astype('int64')
df_upd['end_date'] = df_upd['end_date'].astype('int64')
df_upd['ep_len'] = df_upd['ep_len'].astype('int64')
df_upd['rating'] = df_upd['rating'].astype('float')

df = df_upd.fillna(0)

In [ ]:
# код для удаления года из названия - использовать только если будут проблемы при соединении с пользовательским датасетом

#def name_check(name):
#    return re.sub(r'[(]\d{4}[)]', '', name)

#df['name_jp'], df['name_ru'] = df['name_jp'].apply(name_check), df['name_ru'].apply(name_check)

In [ ]:
all_genres = df['genres'].tolist()
for list in all_genres:
    for i in range(len(list)):
        if list[i] == 'cgdct':
            list[i] = 'милые девушки делают милые вещи'
    list.sort()

df['genres'] = all_genres

In [ ]:
# код для просмотра всех встречающихся жанров

genres = []
for i in all_genres:
    genres.extend(i)

print(sorted(set(genres)))

In [ ]:
df.to_csv('anime_info_full.csv', sep = ',', encoding='utf-8')

## обработка описаний

In [ ]:
df = pd.read_csv('anime_info_full.csv', converters={'genres': pd.eval})

In [ ]:
genres_description = []
genres = df['genres']
description = df['description']

for row in range(len(df)):
    res = ', '.join(genres[row]) + '\t' + '\n'*2 + description[row]
    genres_description.append(res)

df['genres_description'] = genres_description

In [ ]:
# удаление имен из описаний и лемматизация

nlp = stanza.Pipeline(lang='ru', use_gpu=True)

def clean_description(text):
    doc = nlp(text)
    no_names = []
    for sentence in doc.sentences:
        for t in sentence.tokens:
            no_names.append('') if t.ner in ['B-PER', 'E-PER', 'S-PER', 'I-PER'] else no_names.append(t.text)

    not_lemmatized = nlp(' '.join(no_names))
    res = []
    for sent in not_lemmatized.sentences:
        lem_sent = [word.lemma for word in sent.words]
        res.extend(lem_sent)
        res.extend('\n')

    return ' '.join(res)

In [ ]:
sent_transformer = SentenceTransformer('distiluse-base-multilingual-cased-v2')

def sentence_embeddings(text):
    clean_text = clean_description(text)
    tok = nltk.sent_tokenize(clean_text)
    embeddings = sent_transformer.encode(tok)
    return np.mean(embeddings, axis=0)

In [ ]:
df['sentence_embeddings'] = df['genres_description'].apply(sentence_embeddings)

In [ ]:
df.to_csv('anime_info_emb.csv', sep = ',', encoding='utf-8')

In [ ]:
# если нужно открыть файл

#df = pd.read_csv('anime_info_emb.csv')
#df = df.drop('Unnamed: 0', axis=1)

#df['sentence_embeddings'] = df['sentence_embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

## пользовательские данные

In [ ]:
# функция для обработки пользовательского датасета

def concat_data(file_name):
    df_user = pd.read_json(f'{file_name}.json')
    df_user.drop(['target_id', 'target_type', 'rewatches', 'episodes', 'text'], axis=1, inplace=True)
    df_user = df_user[df_user.status != 'planned']
    df_user = df_user[df_user.status != 'on_hold']

    status = df_user['status'].tolist()
    score = df_user['score'].tolist()
    for i in range(len(df_user)):
        if status[i] == 'dropped' and score[i] == 0:
            score[i] = -1
    df_user['score'] = score
    
    df = pd.read_csv('anime_info_emb.csv')
    df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
    df['sentence_embeddings'] = df['sentence_embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

    names_jp = df['name_jp'].tolist()
    score = []
    status = []

    for name in names_jp:
        try:
            df_user.loc[df_user['target_title'] == name]
            score.append(df_user.loc[df_user['target_title'] == name]['score'].item())
            status.append(df_user.loc[df_user['target_title'] == name]['status'].item())
        except:
            score.append(0)
            status.append('not_watched')

    df['user_score'] = score
    df['user_status'] = status
    
    return df

In [ ]:
# similarity_to_user будет учитывать все просмотренные работы

#def user_profile(file_name):
#    df = concat_data(file_name)
#    df_watched = df[df.user_status == 'completed']
#    emb = df_watched['sentence_embeddings'].tolist()
#    return np.mean(emb, axis=0)

In [ ]:
# similarity_to_user будет рассчитана только по работам с оценкой не ниже средней оценки пользователя
# (если просмотрено больше 300 работ и не менее 100 из них оценены)

def user_profile(file_name):
    df = concat_data(file_name)
    df_watched = df[df.user_status == 'completed']
    df_no_zero = df_watched[df_watched['user_score'] > 0]
    
    if len(df_watched) >= 300 and len(df_no_zero) >= 100:
        user_score = df_no_zero['user_score'].tolist()
        average = (sum(user_score) / float(len(user_score))) // 1
        df_high_score = df_watched[df_watched['user_score'] >= average]
        emb = df_high_score['sentence_embeddings'].tolist()
    else:
        emb = df_watched['sentence_embeddings'].tolist()
    
    return np.mean(emb, axis=0)

In [ ]:
def cos_sim(description, user):
  return np.dot(description, user)/(np.linalg.norm(description)*np.linalg.norm(user))

In [ ]:
def similarity_to_user(file_name):
    df = concat_data(file_name)
    user = user_profile(file_name)

    emb = df['sentence_embeddings'].tolist()
    cos_sim_list = []
    for i in range(len(df)):
        res = cos_sim(emb[i], user)
        cos_sim_list.append(res)
    df['cos_sim'] = cos_sim_list

    return df

In [ ]:
# уже на этом этапе можно посмотреть рекомендации, основанные только на косинусном подобии работ и профиля пользователя

df = similarity_to_user('Relax_Iam_Exodium_animes')
df_not_watched = df[df.user_status == 'not_watched']
df_not_watched.sort_values(by=['cos_sim'], ascending=False)[:10]

## предсказание оценки пользователя

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
file_name = 'tuominn_animes'

In [ ]:
df = similarity_to_user(file_name)
df = df.drop(df[(df['start_date'] == 0) & (df['end_date'] == 0)].index)
train = df[df.user_status != 'not_watched']
predict = df[df.user_status == 'not_watched']

In [ ]:
y_train = train['user_score']

X_train = train[['cos_sim']]
X_predict = predict[['cos_sim']]

X_train_not_enc = train[['type', 'age_rating', 'genres', 'studio']]
X_predict_not_enc = predict[['type', 'age_rating', 'genres', 'studio']]

X_train_not_scaled = train[['ep', 'ep_len', 'start_date', 'rating']]
X_predict_not_scaled = predict[['ep', 'ep_len', 'start_date', 'rating']]

In [ ]:
one_hot = OneHotEncoder(handle_unknown='ignore')
one_hot.fit(X_train_not_enc)
X_train_enc = one_hot.transform(X_train_not_enc)
X_test_enc = one_hot.transform(X_predict_not_enc)

scaler = MinMaxScaler()
scaler.fit(X_train_not_scaled)
X_train_scaled = scaler.transform(X_train_not_scaled)
X_predict_scaled = scaler.transform(X_predict_not_scaled)

X_train_transformed = np.concatenate([X_train, X_train_enc.todense(), X_train_scaled], axis=1)
X_predict_transformed = np.concatenate([X_predict, X_test_enc.todense(), X_predict_scaled], axis=1)

In [ ]:
X_train_tensor = torch.tensor(X_train_transformed, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_predict_tensor = torch.tensor(X_predict_transformed, dtype=torch.float32).to(device)

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, 
                        batch_size=10, 
                        shuffle=True)

In [ ]:
class NonLinearRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NonLinearRegressionModel, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)

        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        out = self.fc4(out)

        return out

In [ ]:
input_dim = X_train_transformed.shape[1]
hidden_dim = input_dim * 2
output_dim = 1
num_epochs = 100

model = NonLinearRegressionModel(input_dim, hidden_dim, output_dim)
model.to(device)

loss_function = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, loss_function, optimizer, train_loader, num_epochs):
    losses = []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        losses.append(running_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    return losses

In [ ]:
plt.plot(train_model(model, loss_function, optimizer, train_loader, num_epochs))
plt.title('Training Loss')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_predict_tensor).cpu().numpy()

predicted_scores = pd.DataFrame(predictions, columns=['predicted_score'])
predict['predicted_score'] = predictions

In [ ]:
res = predict.sort_values(by=['predicted_score'], ascending=False)[:10]
res['name_ru'].tolist()

In [ ]:
predict.sort_values(by=['predicted_score'], ascending=False)[:10]

In [ ]:
predict.sort_values(by=['predicted_score'])[:20]

In [ ]:
res = predict.sort_values(by=['predicted_score'], ascending=False)[:10]
res = res.drop(res[['description', 'link', 'genres_description', 'sentence_embeddings', 'user_status', 'user_score']], axis=1)

In [ ]:
res

## выдача рекомендаций

**вариант 1:** рекомендации с учетом косинусного подобия, предсказанной оценки и оценки с сайта; в рекомендации попадает только то, что похоже на пользователя минимум на заданное значение косинусного подобия

In [ ]:
num = int(input('Сколько рекомендаций вы хотите получить?'))

prediction_scaled = scaler.fit_transform(predict[['rating', 'predicted_score', 'cos_sim']])
prediction_mean = np.mean(prediction_scaled, axis=1)
predict['prediction'] = prediction_mean

cos_sim_input = float(input('Введите минимальное допустимое значение похожести работ на уже просмотренные вами (от 0 до 1):'))
rec = predict[predict['cos_sim'] >= cos_sim_input]
rec.sort_values(by=['prediction'], ascending=False)[:num]

**вариант 2:** рекомендации с учетом косинусного подобия и предсказанной оценки с фильтрацией по оценке с сайта (можно задать минимально допустимую оценку)

In [ ]:
num = int(input('Сколько рекомендаций вы хотите получить?'))

prediction_scaled = scaler.fit_transform(predict[['predicted_score', 'cos_sim']])
prediction_mean = np.mean(prediction_scaled, axis=1)
predict['prediction'] = prediction_mean

rating_input = int(input('Введите минимальное допустимое значение рейтинга:'))
rec = predict[predict['rating'] >= rating_input]
rec.sort_values(by=['prediction'], ascending=False)[:num]

**вариант 3:** самый гибкий и настраиваемый, но требующий больше всего решений от пользователя

In [ ]:
# 1 - предсказанная оценка пользователя
# 2 - рейтинг с сайта
# 3 - ничего из этого (рекомендации будут выданы исходя только из косинусного подобия работ и профиля пользователя)

num = int(input('Сколько рекомендаций вы хотите получить?'))
param_input = [int(i) for i in input('Выберите параметры, которые хотите учитывать для выдачи рекомендаций. Введите цифры через проблел:').strip(' ').split(' ')]
rating_input = 0

rec_param = ['cos_sim']
for j in param_input:
    if j == 1:
        rec_param.append('predicted_score')
    elif j == 2:
        rec_param.append('rating')
        rating_input = int(input('Введите минимальное допустимое значение рейтинга:'))

if 3 not in param_input:
    prediction_scaled = scaler.fit_transform(predict[rec_param])
    prediction_mean = np.mean(prediction_scaled, axis=1)
    predict['prediction'] = prediction_mean
    cos_sim_input = float(input('Введите минимальное допустимое значение похожести работ на уже просмотренные вами (от 0 до 1):'))
    if 2 in param_input:
        rec = predict[(predict['cos_sim'] >= cos_sim_input) & (predict['rating'] >= rating_input)].sort_values(by=['prediction'], ascending=False)[:num]
    else:
        rec = predict[predict['cos_sim'] >= cos_sim_input].sort_values(by=['prediction'], ascending=False)[:num]
else:
    rec = predict.sort_values(by=['cos_sim'], ascending=False)[:num]


rec

**фильтрация по жанрам**: в рекомендации попадают только работы с заданными жанрами

In [ ]:
prediction_scaled = scaler.fit_transform(predict[['rating', 'predicted_score', 'cos_sim']])
prediction_mean = np.mean(prediction_scaled, axis=1)
predict['prediction'] = prediction_mean


rec = predict[predict['cos_sim'] >= cos_sim_input]
cos_sim_input = float(input('Введите минимальное допустимое значение похожести работ на уже просмотренные вами (от 0 до 1):'))
filter_genres = input('Введите жанры через запятую:')

def contains_genre(list, filter_input):
    if ',' in filter_input:
        input_list = [word.strip() for word in filter_input.split(',')]
        return all(word in list for word in input_list)
    else:    
        return filter_input in list

filtered_df = rec[rec['genres'].apply(contains_genre, filter_input=filter_genres)]
filtered_df.sort_values(by=['prediction'], ascending=False)[:10]

## поиск по описанию

In [ ]:
df = pd.read_csv('anime_info_emb.csv')
df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
df['sentence_embeddings'] = df['sentence_embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

In [ ]:
# описания чего-то конкретного

#text = 'аниме про японского школьника, который убивал преступников с помощью тетради, которую ему дал бог смерти, и детектива, который пытался его поймать'
#text = 'аниме про двух братьев, которые начали заниматься алхимией, чтобы воскресить маму, но в итоге один потерял все тело, а второй ногу и руку'
#text = 'аниме про клуб бегунов, которые собрались пробежать марафон'
text = 'аниме про четырех друзей-школьников. просто повседневность комедия про школу'

check = sentence_embeddings(text)

df['result'] = df['sentence_embeddings'].apply(cos_sim, user=check)
df.sort_values(by=['result'], ascending=False)[:5]

In [ ]:
# описания с предпочтениями, можно снова задать минимальную допустимую оценку

text = 'аниме про спортивные соревнования'
#text = 'повседневность иясикэй про работу'
#text = 'повседневность комедия про школу'
#text = 'повседневность комедия про школу без фэнтези'

check = sentence_embeddings(text)

num = int(input())
df_emb = df[df['rating'] >= num]

df_emb['result'] = df_emb['sentence_embeddings'].apply(cos_sim, user=check)
df_emb.sort_values(by=['result'], ascending=False)[:5]